## **Navier Stokes equations**

This tutorial is freely inspired by the [RBniCS tutorial on Navier-Stokes equations](https://colab.research.google.com/github/RBniCS/RBniCS/blob/open-in-colab/tutorials/17_navier_stokes/tutorial_navier_stokes_1_deim.ipynb). 
We focus on the two-dimensional backward-facing step domain $\Omega$, defined as follows.

<img src="https://github.com/RBniCS/RBniCS/raw/master/tutorials/17_navier_stokes/data/backward_facing_step.png" width="80%"/>

A non-homogeneous inlet Dirichlet boundary is applied on the left of the domain. No-slip boundaries (i.e. zero velocity) is imposed on the top and on the bottom of $\Omega$. The right boundary features homogeneous Neumann conditions.

The inflow velocity is parametric
$$\boldsymbol{u}(\boldsymbol{x};\mu)=\mu\left(\frac{1}{2.25}(x_1-2)(5-x_1),0\right ) \quad \forall \boldsymbol{x} \in \Omega.$$ 

The parameter $\mu$, characterizes the inlet velocity and the **Reynolds number** (i.e. the indicator of how much convective is the flow). The parameter space is $\mathcal{P}=[1.0,80.0].$



To solve the problem, we rely on [FEniCS library](https://fenicsproject.org/download/archive/). FEniCS is a python-based library that deals with the Finite Element approximation of PDE.

In [ ]:
# Install FEniCS
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

**Parametric problem**

Let us denote with $\boldsymbol{u}(\mu)$ and $p(\mu)$ the velocity and the pressure variables of the system.

The weak formulation of Navier-Stokes equation reads: for a given parameter $\mu \in \mathcal{P},$ find $\boldsymbol u(\mu) \in \mathbb{V}, \; p \in\mathbb{M}$ such that </center>
$$
    \begin{cases}
        \nu \displaystyle \int_{x} \nabla \boldsymbol{u} : \nabla \boldsymbol{v} \ dx + \int_{\Omega} [(\boldsymbol{u} \cdot \nabla) \boldsymbol{u}] \cdot \boldsymbol{v} \ dx - \int_{\Omega} p \nabla \cdot \boldsymbol{v} \ dx = \int_{\Omega} \boldsymbol{f} \cdot \boldsymbol{v} \ dx, \quad \forall \boldsymbol{v} \in\mathbb{V},  \\
        \displaystyle \int_{\Omega} q \nabla \cdot \boldsymbol{u} \ dx = 0, \quad \forall q \in\mathbb{M}
    \end{cases}
$$
    
where

* $\nu$ is kinematic viscosity,
* $\mathbb{V}=[H^1_{\Gamma_{wall}}(\Omega)]^2$,
* $\mathbb{M}=L^2(\Omega)$ (we have an outlet, so no problems with the uniqueness of the pressure).

As in the Stokes case, the well-posedness is related on the inf-sup condition of the divergence operator.
Thus: we need $\mathbb P^2 - \mathbb P^1$ elements at the FOM level and we need supremizer stabilization for the ROM simulations.


Let us start to understand how to perform a FE simulation with FEniCS!

Let us read the mesh!

In [ ]:
# Download data files
!mkdir -p data
![ -f data/backward_facing_step.xml ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/17_navier_stokes/data/backward_facing_step.xml -O data/backward_facing_step.xml
![ -f data/backward_facing_step_facet_region.xml ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/17_navier_stokes/data/backward_facing_step_facet_region.xml -O data/backward_facing_step_facet_region.xml
![ -f data/backward_facing_step_physical_region.xml ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/17_navier_stokes/data/backward_facing_step_physical_region.xml -O data/backward_facing_step_physical_region.xml

Let us import FEniCS and the mesh.
We define some parameters for the nonlinear solver.

In [ ]:
from dolfin import *

In [ ]:
mesh = Mesh("data/backward_facing_step.xml")
subdomains = MeshFunction("size_t", mesh, "data/backward_facing_step_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/backward_facing_step_facet_region.xml")

In [ ]:
# Prepare nonlinear solver
snes_solver_parameters = {"nonlinear_solver": "snes",
                          "snes_solver": {"linear_solver": "umfpack",
                                          "maximum_iterations": 50,
                                          "report": True,
                                          "error_on_nonconvergence": True}}


Let us define the mixed space $\mathbb W = \mathbb V \times \mathbb M$. Just below you find the definition of trial and test functions.

In [ ]:
# Define function space (P2-P1)
V_element = VectorElement("Lagrange", mesh.ufl_cell(), 2)
M_element = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
W_element = MixedElement(V_element, M_element)
W = FunctionSpace(mesh, W_element)


In [ ]:
# Define trial and test functions
delta_up = TrialFunction(W)
vq = TestFunction(W)
up = Function(W)

(delta_u, delta_p) = split(delta_up)
(v, q) = split(vq)
(u, p)= split(up)


Below we define the parameter, the inlet expression and the boundaries conditions.

In [ ]:
mu = 2.
inlet = Expression(("mu*1. / 2.25 * (x[1] - 2) * (5 - x[1])", "0."), mu=mu, degree=2)

# Impose boundary conditions
bc = [DirichletBC(W.sub(0), inlet, boundaries, 1),
                   DirichletBC(W.sub(0), Constant((0.0, 0.0)), boundaries, 2)]

We solve the problem by mean of the ``dolfin`` nonlinear solver. First we define the residual ``F``, then the Jacobian.

In [ ]:

# Define PDE and problem
F = (inner(grad(u), grad(v))*dx + inner(grad(u)*u, v)*dx - div(v)*p*dx 
    + div(u)*q*dx)

J = derivative(F, up, delta_up)

problem = NonlinearVariationalProblem(F, up, bc, J)
solver  = NonlinearVariationalSolver(problem)
solver.parameters.update(snes_solver_parameters)




**What does the Newton's solver do?**

Let us call the global mixed solution variable $U_{\delta} = \begin{bmatrix}
\mathbf u_{\delta}\\
p_{\delta}
\end{bmatrix}$.


Setting $\delta U_{\delta} = U^{k+1}_{\delta} - U^{k}_{\delta}$, we want to solve 
$$
J(U^{k}_{\delta};\mu) \delta U_\delta = - F(U^{k}_{\delta}; \mu).
$$
Namely, we  need:
1. the Jacobian matrix $J(U^{k}_\delta;\mu) \in \mathbb R^{(N_{\delta}^u + N_{\delta}^p) \times (N_{\delta}^u + N_{\delta}^p)}$ is
$$
J(U^{k}_\delta;\mu) =
\begin{bmatrix}
A + C_1(\mathbf u_{\delta}^{k}) + C_2(\mathbf u_{\delta}^{k}) & B^T \\
B & 0 \\ 
\end{bmatrix},
$$
where $A$ and $B$ are the classicla stiffness matrix and divergence matrix, respectively, while

$$
C_1(\mathbf u_{\delta}^{k})_{ij} = \int_{\Omega} [(\boldsymbol{u}_{\delta}^k \cdot \nabla) \boldsymbol{\varphi}_j] \cdot \boldsymbol{\varphi}_i \ dx \quad \text{and} \quad 
C_2(\mathbf u_{\delta}^{k})_{ij} = \int_{\Omega} [(\boldsymbol{\varphi}_j\cdot \nabla) \boldsymbol{u}_{\delta}^k ] \cdot \boldsymbol{\varphi}_i \ dx,
$$
where $\{\boldsymbol \varphi\}_{i}^{N_{\delta}^u}$ is a FE basis for the velocity.
2. The residual vector
$$
F(U^{k}_{\delta}; \mu) = \begin{bmatrix}
A \boldsymbol  u_{\delta}^{k} + C_1(\mathbf u_{\delta}^{k})\boldsymbol  u_{\delta}^{k} + B^T p_{\delta}^{k} - \boldsymbol f  \\
B \boldsymbol u_{\delta}^{k} \\ 
\end{bmatrix}.
$$

One goes on until a convergence criterium is met (ugually on the norm of the residual vector).


In [ ]:

solver.solve()
(u, p) = up.split(True)


Let us plot velocity and pressure fields.

In [ ]:
plot(u, title='Velocity')


In [ ]:
plot(p, title='Pressure')

**What about ROMs?**

To build the reduced approximation we rely on a POD approximation with supremizer stabilization (inexact, exactly as Stokes problem).

We call the two basis matrices as $\mathbb B_{us}$ and $\mathbb B_p$.

To build the spaces and to perform online phase, we rely on [RBniCS](https://www.rbnicsproject.org/).

RBniCS is a library based on FEniCS that deals with a lot parametric problems and structures.

Let us install it and import it.

In [ ]:
# Install RBniCS
try:
    import rbnics
except ImportError:
    !pip3 install git+https://github.com/RBniCS/RBniCS.git
    import rbnics
import rbnics.utils.config
assert "dolfin" in rbnics.utils.config.config.get("backends", "required backends")

In [ ]:
from ufl import transpose
from rbnics import *

**Affine Decomposition**

In this example we use DEIM approximation to recover the affine assumption over Navier-Stokes equation. 

Indeed, for nonliner problems as such, without any kind of _hyper-reduction_ we will deal with something of the form

$$
F_N(U_N;\mu) = \mathbb B^T F(\mathbb B U_N)
$$
for the residual vector and 
$$
J_N(U_N; \mu) = \mathbb B^T J(\mathbb B U_N; \mu) \mathbb B
$$
for the Jacobian matrix.

At each iteration the system to be solved is

$$
\mathbb B^T J(\mathbb B U_N^{k};\mu) \mathbb B \delta U_N = - \mathbb B^T F(\mathbb B U_N^{k}; \mu).
$$

Namely at **each evaluation of the Newton's method you have to project back the reduced solution at iteration $k$ and assemble the residual and the Jacobian**.

DEIM (it is like EIM with POD) helps since the residual and the Jacobian are approximated, respectively, as
$$
F_N(U_N;\mu) \approx \sum_{q=1}^{Q_q}a_F^q(U_N, \mu)\mathbb B^T F^q,
$$
and 
$$
F_J(U_N;\mu) \approx \sum_{q=1}^{Q_q}\lambda_J^q(U_N, \mu)\mathbb B^T J^q \mathbb B.
$$
In this way all the vectors $F^q$ and matrices $J^q$ can be pre-assembled.

Let us see how RBniCS works with it. It uses a DEIM approximation online while uses standard snapshots collection in the offline phase (decorators).

In [ ]:
@DEIM("online", basis_generation="Greedy")
@ExactParametrizedFunctions("offline")
class NavierStokes(NavierStokesProblem):

    # Default initialization of members
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        NavierStokesProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        dup = TrialFunction(V)
        (self.du, self.dp) = split(dup)
        (self.u, _) = split(self._solution)
        vq = TestFunction(V)
        (self.v, self.q) = split(vq)
        self.dx = Measure("dx")(subdomain_data=self.subdomains)
        self.ds = Measure("ds")(subdomain_data=self.boundaries)
        #
        self.inlet = Expression(("1. / 2.25 * (x[1] - 2) * (5 - x[1])", "0."), degree=2)
        self.f = Constant((0.0, 0.0))
        self.g = Constant(0.0)
        # Customize nonlinear solver parameters
        self._nonlinear_solver_parameters.update({
            "linear_solver": "mumps",
            "maximum_iterations": 20,
            "report": True
        })

    # Return custom problem name
    def name(self):
        return "NavierStokesDEIM1"

    # Return theta multiplicative terms of the affine expansion of the problem.
    @compute_theta_for_derivatives
    @compute_theta_for_supremizers
    def compute_theta(self, term):
        mu = self.mu
        if term == "a":
            theta_a0 = 1.
            return (theta_a0,)
        elif term in ("b", "bt"):
            theta_b0 = 1.
            return (theta_b0,)
        elif term == "c":
            theta_c0 = 1.
            return (theta_c0,)
        elif term == "f":
            theta_f0 = 1.
            return (theta_f0,)
        elif term == "g":
            theta_g0 = 1.
            return (theta_g0,)
        elif term == "dirichlet_bc_u":
            theta_bc00 = mu[0]
            return (theta_bc00,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    @assemble_operator_for_derivatives
    @assemble_operator_for_supremizers
    def assemble_operator(self, term):
        dx = self.dx
        if term == "a":
            u = self.du
            v = self.v
            a0 = inner(grad(u) + transpose(grad(u)), grad(v)) * dx
            return (a0,)
        elif term == "b":
            u = self.du
            q = self.q
            b0 = - q * div(u) * dx
            return (b0,)
        elif term == "bt":
            p = self.dp
            v = self.v
            bt0 = - p * div(v) * dx
            return (bt0,)
        elif term == "c":
            u = self.u
            v = self.v
            c0 = inner(grad(u) * u, v) * dx
            return (c0,)
        elif term == "f":
            v = self.v
            f0 = inner(self.f, v) * dx
            return (f0,)
        elif term == "g":
            q = self.q
            g0 = self.g * q * dx
            return (g0,)
        elif term == "dirichlet_bc_u":
            bc0 = [DirichletBC(self.V.sub(0), self.inlet, self.boundaries, 1),
                   DirichletBC(self.V.sub(0), Constant((0.0, 0.0)), self.boundaries, 2)]
            return (bc0,)
        elif term == "inner_product_u":
            u = self.du
            v = self.v
            x0 = inner(grad(u), grad(v)) * dx
            return (x0,)
        elif term == "inner_product_p":
            p = self.dp
            q = self.q
            x0 = inner(p, q) * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")


# Customize the resulting reduced problem
@CustomizeReducedProblemFor(NavierStokesProblem)
def CustomizeReducedNavierStokes(ReducedNavierStokes_Base):
    class ReducedNavierStokes(ReducedNavierStokes_Base):
        def __init__(self, truth_problem, **kwargs):
            ReducedNavierStokes_Base.__init__(self, truth_problem, **kwargs)
            self._nonlinear_solver_parameters.update({
                "report": True,
                "line_search": "wolfe"
            })

    return ReducedNavierStokes

**The Main**

Before we were defining the problem class. Now we are ready to call the methods to solve the parametric problem.

Let us start with the mesh!

In [ ]:
mesh = Mesh("data/backward_facing_step.xml")
subdomains = MeshFunction("size_t", mesh, "data/backward_facing_step_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/backward_facing_step_facet_region.xml")

Let us create the Taylor-Hooed approximation!

In [ ]:
element_u = VectorElement("Lagrange", mesh.ufl_cell(), 2)
element_p = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
element = MixedElement(element_u, element_p)
V = FunctionSpace(mesh, element, components=[["u", "s"], "p"])

We are now ready to allocate the Navier-Stokes class.

In [ ]:
problem = NavierStokes(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(1.0, 80.0)]
problem.set_mu_range(mu_range)

We have to set the reduction strategy, i.e. POD.

In [ ]:
reduction_method = PODGalerkin(problem)
reduction_method.set_Nmax(10, DEIM=20)

**We perform the offline phase!**

In [ ]:
lifting_mu = (1.0,)
problem.set_mu(lifting_mu)
reduction_method.initialize_training_set(100, DEIM=144, sampling=EquispacedDistribution())
reduced_problem = reduction_method.offline()

**Perform an online solve**

In [ ]:
online_mu = (10.0,)
reduced_problem.set_mu(online_mu)
reduced_solution = reduced_problem.solve()

In [ ]:
plot(reduced_solution, reduced_problem=reduced_problem, component="u")

In [ ]:
plot(reduced_solution, reduced_problem=reduced_problem, component="p")

**Error Analysis and Speedup Analysis**

In [ ]:
reduction_method.initialize_testing_set(16, DEIM=25, sampling=EquispacedDistribution())
reduction_method.error_analysis()

In [ ]:
reduction_method.speedup_analysis()